In [1]:
## Import required libraries
import requests
import pandas as pd
from IPython.display import JSON

In [3]:
# call data from city bikes API
citybikes_network = requests.get("http://api.citybik.es/v2/networks").json()

# Take a look at the returned Json in a cleaner format
JSON(citybikes_network)



<IPython.core.display.JSON object>

In [5]:
## After Identifying where the info we need is stored, create variable x, just to make it easier to work with
x = citybikes_network['networks']

In [7]:
## Search the output for our chosen city and get the network id/ids
## we have 615 networks, so loop through them and find an output for the city Vancouver
for i in range(615):
    if x[i]['location']['city'] == 'Vancouver':
        print (i)

241


In [9]:
## Identify the network id for Vancouver
print(x[241]['id'])

mobibikes


In [13]:
## Now that we have the network id for the city we want, send another call to get data about the bike stations in the city
network_id = 'mobibikes'
n_json = requests.get("http://api.citybik.es/v2/networks/"+network_id).json()
## Look through the info and identify where we can get the information we want
JSON(n_json)

<IPython.core.display.JSON object>

In [19]:
## Isolate the data we are interested in
station_info = n_json['network']['stations']
## Load into dataframe
type(station_info)
stations = pd.DataFrame(station_info)
stations

,empty_slots,extra,free_bikes,id,latitude,longitude,name,timestamp
0,14,"{'ebikes': 4, 'has_ebikes': True, 'last_update...",21,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10th & Cambie,2023-02-13T19:22:31.595000Z
1,7,"{'ebikes': 1, 'has_ebikes': True, 'last_update...",9,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,Yaletown-Roundhouse Station,2023-02-13T19:22:31.591000Z
2,13,"{'ebikes': 0, 'has_ebikes': True, 'last_update...",13,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,Dunsmuir & Beatty,2023-02-13T19:22:31.592000Z
3,14,"{'ebikes': 0, 'has_ebikes': True, 'last_update...",2,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,12th & Yukon (City Hall),2023-02-13T19:22:31.593000Z
4,7,"{'ebikes': 2, 'has_ebikes': True, 'last_update...",9,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,8th & Ash,2023-02-13T19:22:31.604000Z
...,...,...,...,...,...,...,...,...
236,11,"{'ebikes': 2, 'has_ebikes': True, 'last_update...",11,d8c4fe5e09b6f010fbaa85cc7302d66b,49.245535,-123.120496,Heather & 29th,2023-02-13T19:22:31.519000Z
237,10,"{'ebikes': 0, 'has_ebikes': True, 'last_update...",16,0c42f45e4a14957ad4a6d521d0ba8bc3,49.289255,-123.132677,Cardero & Robson,2023-02-13T19:22:31.428000Z
238,17,"{'ebikes': 0, 'has_ebikes': True, 'last_update...",0,62a202eb98c0a4b09ba6746f5ee94ab3,49.252656,-123.067965,Commercial & 20th,2023-02-13T19:22:31.516000Z
239,21,"{'ebikes': 0, 'has_ebikes': True, 'last_update...",2,a6c6df138cbc97b9f81e7ffbbebdde6f,49.277527,-123.129464,Hornby & Drake,2023-02-13T19:22:31.540000Z


In [22]:
## Keep only the information we are interested in
stations_clean = stations[['id','free_bikes','latitude','longitude']].rename(columns = {'free_bikes':'num_bikes','latitude':'lat', 'longitude':'long'})
stations_clean

,id,num_bikes,lat,long
0,7a19c49f486d7c0c02b3685d7b240448,21,49.262487,-123.114397
1,32603a87cfca71d0f7dfa3513bad69d5,9,49.274566,-123.121817
2,6d42fa40360f9a6b2bf641c7b8bb2862,13,49.279764,-123.110154
3,66f873d641d448bd1572ab086665a458,2,49.260599,-123.113504
4,485d4d24c803cfde829ab89699fed833,9,49.264215,-123.117772
...,...,...,...,...
236,d8c4fe5e09b6f010fbaa85cc7302d66b,11,49.245535,-123.120496
237,0c42f45e4a14957ad4a6d521d0ba8bc3,16,49.289255,-123.132677
238,62a202eb98c0a4b09ba6746f5ee94ab3,0,49.252656,-123.067965
239,a6c6df138cbc97b9f81e7ffbbebdde6f,2,49.277527,-123.129464


In [24]:
## Add a new column 'll' containing the combined lat and long, this will probably be helpful when working with Foursquare and Yelp API 
stations_clean['ll'] = stations_clean['lat'].astype(str) + ',' + stations_clean['long'].astype(str)
stations_clean

,id,num_bikes,lat,long,ll
0,7a19c49f486d7c0c02b3685d7b240448,21,49.262487,-123.114397,"49.262487,-123.114397"
1,32603a87cfca71d0f7dfa3513bad69d5,9,49.274566,-123.121817,"49.274566,-123.121817"
2,6d42fa40360f9a6b2bf641c7b8bb2862,13,49.279764,-123.110154,"49.279764,-123.110154"
3,66f873d641d448bd1572ab086665a458,2,49.260599,-123.113504,"49.260599,-123.113504"
4,485d4d24c803cfde829ab89699fed833,9,49.264215,-123.117772,"49.264215,-123.117772"
...,...,...,...,...,...
236,d8c4fe5e09b6f010fbaa85cc7302d66b,11,49.245535,-123.120496,"49.245535,-123.120496"
237,0c42f45e4a14957ad4a6d521d0ba8bc3,16,49.289255,-123.132677,"49.289255,-123.132677"
238,62a202eb98c0a4b09ba6746f5ee94ab3,0,49.252656,-123.067965,"49.252656,-123.067965"
239,a6c6df138cbc97b9f81e7ffbbebdde6f,2,49.277527,-123.129464,"49.277527,-123.129464"


In [25]:
''''''''

''

In [29]:
## Test single request
url = "https://api.foursquare.com/v3/places/search"

params = {
  	"categories": "13030",
  	"ll": "49.262487,-123.114397",
    'radius': "1000",
  	"sort":"DISTANCE",
    "limit":"50"
}

headers = {
    "Accept": "application/json",
    "Authorization": key
}

response = requests.request("GET", url, params=params, headers=headers)
res = response.json()
type(res)

dict

In [30]:
JSON(res)

<IPython.core.display.JSON object>

In [38]:
## we want to create a list of fsq ids
bar_id_list = []
def get_bar_ids(coords):
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "categories": "13003",
        "ll": str(coords),
        'radius': "1000",
        "sort":"DISTANCE",
        "limit":"50"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": key
    }
## Get the request
    response = requests.request("GET", url, params=params, headers=headers)
## Convert to python object with .json()
    res = response.json()
    ## Create loop to pull out fsq_ids
    
    for i in range(len(res['results'])):
        bar_id_list.append(res['results'][i]['fsq_id'])
        

In [39]:
## Apply the function to our column of coord, the 'll' column from stations_clean dataframe
stations_clean['ll'].apply(get_bar_ids)

0      None
1      None
2      None
3      None
4      None
       ... 
236    None
237    None
238    None
239    None
240    None
Name: ll, Length: 241, dtype: object

In [43]:
len(bar_id_list)

6545

In [44]:
## Remove duplicates from the list
bar_id_list_clean = list(set(bar_id_list))

In [45]:
## Looks like it worked. We now have a unique list of all the bars in a 1km radius from each of our bike stations!
len(bar_id_list_clean)

373

In [78]:
## Run a test query for the bar details


In [81]:
## To call for specific information we want 
url = "https://api.foursquare.com/v3/places/5358725f498ea4226bba2e18"

params = {
  	"session_token": "YOUR USER GENERATED SESSION TOKEN HERE",
    "fields": "fsq_id,name,rating,stats,popularity,price"
}

headers = {
    "Accept": "application/json",
    "Authorization": key
}

response = requests.request("GET", url, params=params, headers=headers)
res = response.json()

In [88]:
bar_info_list = []
for f_id in bar_id_list_clean:
    url = "https://api.foursquare.com/v3/places/" + f_id

    params = {
        "session_token": "YOUR USER GENERATED SESSION TOKEN HERE",
        "fields": "fsq_id,name,rating,stats,popularity,price"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": key
    }

    response = requests.request("GET", url, params=params, headers=headers)
    res = response.json()
    bar_info_list.append(res)

In [89]:
JSON(bar_info_list)

<IPython.core.display.JSON object>

In [92]:
df_bar_foursquare = pd.json_normalize(bar_info_list)

In [93]:
## we now have a dataframe of bar info from bars in a 1km radius from bike location from foursquare
df_bar_foursquare

,fsq_id,name,popularity,rating,stats.total_photos,stats.total_ratings,stats.total_tips,price
0,5358725f498ea4226bba2e18,Vancouver Urban Winery,0.843229,8.3,46.0,33.0,9.0,NaN
1,5840e86d44689a5eabddf770,The American,0.977137,7.6,15.0,29.0,7.0,2.0
2,4aa8712bf964a520505120e3,Score on Davie,0.973307,8.5,252.0,227.0,42.0,2.0
3,4aabe6baf964a520cc5a20e3,Red Room Ultra Bar,0.950636,6.5,116.0,67.0,22.0,2.0
4,53aa2de5498e191dfd681c59,The Lido,0.931449,7.0,12.0,13.0,1.0,2.0
...,...,...,...,...,...,...,...,...
368,557a3846498ee12e5ed1f7e3,The Heatley,0.949000,6.3,18.0,20.0,2.0,2.0
369,5e3e360cb882400008faa264,Bar Corso,0.903158,NaN,NaN,NaN,NaN,3.0
370,4aa734e0f964a520454c20e3,Narrow Lounge,0.952176,7.8,71.0,101.0,24.0,3.0
371,4e4f15981f6eae4d2515db5d,Reflections,0.961664,7.2,170.0,69.0,21.0,3.0


In [ ]:
49.262487	-123.114397

In [98]:
## Yelp call
url = "https://api.yelp.com/v3/businesses/search?latitude=49.262487&longitude=-123.114397&radius=1000&categories=bars&sort_by=best_match&limit=50"

headers = {
    "accept": "application/json",
    "Authorization": ykey
}

response = requests.get(url, headers=headers)

res = response.json()

In [99]:
JSON(res)

<IPython.core.display.JSON object>

In [120]:
##Create list of latitudes and longitudes
lat_list = stations_clean['lat'].values.tolist()
long_list = stations_clean['long'].values.tolist()


In [140]:
yelp_list = []
for lat,long in zip(lat_list,long_list):
    url = "https://api.yelp.com/v3/businesses/search?latitude=" + str(lat) + "&longitude=" + str(long) + "&radius=1000&categories=bars&sort_by=best_match&limit=50"

    headers = {
        "accept": "application/json",
        "Authorization": ykey
    }

    response = requests.get(url, headers=headers)

    res = response.json()
    for i in range(len(res['businesses'])):
        yelp_list.append(res['businesses'][i]['id'])

In [123]:
## Create new list without duplicates
yelp_list_clean = list(set(yelp_list))

In [125]:
len(yelp_list_clean)

318

In [132]:
yelp_info_list = []

for y_id in yelp_list_clean:
    url = "https://api.yelp.com/v3/businesses/" + str(y_id) + "?locale=en_CA"

    headers = {
        "accept": "application/json",
        "Authorization": ykey
    }

    response = requests.get(url, headers=headers)
    res = response.json()
    yelp_info_list.append(res)


In [133]:
JSON(yelp_info_list)

<IPython.core.display.JSON object>

In [134]:
df_bar_yelp = pd.json_normalize(yelp_info_list)

In [136]:
df_bar_yelp.head()

,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,...,location.zip_code,location.country,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude,messaging.url,messaging.use_case_text,price
0,QbvMaN8JekZh9HWRn9e0LA,addah-vancouver,Addah,https://s3-media1.fl.yelpcdn.com/bphoto/G6zLUQ...,True,False,https://www.yelp.ca/biz/addah-vancouver?adjust...,+12365218777,(236) 521-8777,4,...,V6A 2Z6,CA,BC,"[555 Gore Avenue, Vancouver, BC V6A 2Z6]",,49.279690,-123.097520,https://www.yelp.ca/raq/QbvMaN8JekZh9HWRn9e0LA...,Message the Business,NaN
1,Mfdo3zuB2P0xFQ40DLPaEQ,la-quercia-vancouver,La Quercia,https://s3-media4.fl.yelpcdn.com/bphoto/gaMUTV...,True,False,https://www.yelp.ca/biz/la-quercia-vancouver?a...,+16046761007,(604) 676-1007,200,...,V6R 1P2,CA,BC,"[3689 W 4th Avenue, Vancouver, BC V6R 1P2]",,49.268765,-123.185360,NaN,NaN,$$$
2,kZN_-zju67rEQVUSaewRhA,brix-and-mortar-vancouver,Brix & Mortar,https://s3-media4.fl.yelpcdn.com/bphoto/xYkmIr...,True,False,https://www.yelp.ca/biz/brix-and-mortar-vancou...,+16049159463,(604) 915-9463,316,...,V6B 5P6,CA,BC,"[1137 Hamilton Street, Vancouver, BC V6B 5P6]",,49.275616,-123.121910,NaN,NaN,$$$
3,ulsLlVcGvuLOSN0n3bR2zg,harbour-oyster-bar-vancouver,Harbour Oyster + Bar,https://s3-media4.fl.yelpcdn.com/bphoto/h4L5oB...,True,False,https://www.yelp.ca/biz/harbour-oyster-bar-van...,+16042516900,(604) 251-6900,70,...,V5L 3X9,CA,BC,"[1408 Commercial Drive, Vancouver, BC V5L 3X9]",,49.271895,-123.069182,NaN,NaN,$$
4,vXTo69v2bGEyuBdT0X1hrw,la-taqueria-pinche-taco-shop-vancouver-2,La Taqueria Pinche Taco Shop,https://s3-media3.fl.yelpcdn.com/bphoto/tevD3s...,True,False,https://www.yelp.ca/biz/la-taqueria-pinche-tac...,+16045684406,(604) 568-4406,514,...,V8B 1K6,CA,BC,"[322 W Hastings Street, Vancouver, BC V8B 1K6]",,49.282912,-123.110593,NaN,NaN,$$


In [137]:
df_bar_yelp.columns

Index(['id', 'alias', 'name', 'image_url', 'is_claimed', 'is_closed', 'url',
       'phone', 'display_phone', 'review_count', 'categories', 'rating',
       'photos', 'hours', 'transactions', 'location.address1',
       'location.address2', 'location.address3', 'location.city',
       'location.zip_code', 'location.country', 'location.state',
       'location.display_address', 'location.cross_streets',
       'coordinates.latitude', 'coordinates.longitude', 'messaging.url',
       'messaging.use_case_text', 'price'],
      dtype='object')

In [138]:
## Only keep columns we need
df_yelp = df_bar_yelp[['name','review_count', 'rating', 'price']]

In [139]:
df_yelp

,name,review_count,rating,price
0,Addah,4,4.5,NaN
1,La Quercia,200,4.0,$$$
2,Brix & Mortar,316,4.0,$$$
3,Harbour Oyster + Bar,70,4.0,$$
4,La Taqueria Pinche Taco Shop,514,4.5,$$
...,...,...,...,...
313,Moose's Down Under,141,4.0,$$
314,The Vancouver Fish Company,249,3.5,$$$
315,Unwind @ Holiday Inn,7,3.0,$$
316,The Beaver,20,3.5,$
